In [2]:
import os
import sys
import re

STAGE_DIR = 'data/staging'
LOG_DIR = 'data/rab/logs'

In [3]:
edit_ptn = re.compile(
    '^[A-Z]+ '
    '(?P<date_time>\d{4}-\d{1,2}-\d{1,2} \d{2}:\d{2}:\d{2}),.+')

sesh_ptn = re.compile(
    '^[A-Z]+ '
    '(?P<date_time>\d{4}-\d{1,2}-\d{1,2} \d{2}:\d{2}:\d{2}),.+'
    'Session created for (?P<session>\w{2,})$')

In [12]:
add_by_id = ("confirmed add by ID", "pub_by_id")
approve_pub = ("Approving pub", "pub_by_harvest")
cv_upload = ("CV file uploaded", "cv")
pub_form = ("Publication form additions", "pub_manual")
field_edit = ("Adding:", "field_data")
session = ("Session created", "session")
cred_edit = ("Credential form additions:", "form_data")
train_edit = ("Training form additions:", "form_data")
appt_edit = ("Appointment form additions:", "form_data")

log_tests = [ session, add_by_id, approve_pub, cv_upload, pub_form, field_edit,
            cred_edit, train_edit, appt_edit ]

In [13]:
def parse_logline(line, tests):
    matched = [ t[1] for t in tests if t[0] in line ]
    if len(matched) > 1:
        print(line)
    if len(matched) != 1:
        return False
    return matched[0]

In [14]:
with open(os.path.join(STAGE_DIR,'logs','vmgr_edits.csv'), 'w') as f_edits, \
    open(os.path.join(STAGE_DIR,'logs','vmgr_sessions.csv'), 'w') as f_sesh, \
    open(os.path.join(STAGE_DIR,'logs','vmgr_log_errs.txt'), 'w') as f_errs, \
    open(os.path.join(LOG_DIR,'vivo_all.log'), 'r') as f_log:
    
    while True:
        line = f_log.readline()
        if not line:
            break
        tested = parse_logline(line, log_tests)
        if tested == 'session':
            m = re.match(sesh_ptn, line)
        elif tested:
            m = re.match(edit_ptn, line)
        else:
            continue
        if not m:
            f_errs.write(line)
            continue
        
        matches = m.groupdict()
        if matches.get('session'):
            f_sesh.write(
                '{},{}\n'.format(matches['date_time'], matches['session']))
        elif matches.get('date_time'):
            f_edits.write(
                '{},{}\n'.format(matches['date_time'], tested))